# PRODUCTIVIZAR UN MODELO - EJEMPLO H2O

* Vamos a crear un modelo en H2O sobre los datos de Titanic.
* Expondremos una función de predicción a la que pasaremos nuevos datos.
* Que nos devolverá una predicción.

### Importamos Librerías

In [ ]:
import pandas as pd
import h2o
from h2o.estimators.xgboost import H2OXGBoostEstimator
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from flask import Flask, request, jsonify

import warnings
warnings.filterwarnings("ignore")

### Inicializamos la aplicación de Flask

In [ ]:
app = Flask(__name__)

### Modelo

In [ ]:
titanic_df = pd.read_csv('../data/titanic.csv')

In [ ]:
titanic_df.head()

In [ ]:
# Inicializamos el cluster de H2O
h2o.init()

In [ ]:
# Convertimos el dataframe de pandas a un objeto H2O
hf = h2o.H2OFrame(titanic_df)

In [ ]:
hf.head()

In [ ]:
# Convertimos la variable objetivo en factor de otra forma el problema lo entiende como de regresión.
hf['Survived'] = hf['Survived'].asfactor()

In [ ]:
# Dividimos los datos en train y test
train, test = hf.split_frame(ratios=[0.8])

# Definimos las variables predictoras y la variable objetivo
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
target = 'Survived'

In [ ]:
# Definimos el modelo
model = H2OXGBoostEstimator(ntrees=100, max_depth=3, learn_rate=0.1, seed=1)

# Entrenamos el modelo
model.train(x=features, y=target, training_frame=train)

## Apificamos la llamada al modelo para predecir

In [ ]:
# Defino la ruta
@app.route('/predigo', methods=['POST'])
def predigo():
    # Consigo los datos de la petición
    data = request.get_json(force=True)

    # Los convierto a objeto de h2o
    hf_test = h2o.H2OFrame(data)

    # Hago la predicición
    preds = model.predict(hf_test)

    # Convierto la predicción a pandas dataframe
    preds_df = preds.as_data_frame()

    # Renombro la predicción a "result"
    result = preds_df

    # La predicción la convierto a un JSON
    return jsonify(result.to_json(orient='records'))

In [ ]:
# Levanto la aplicación en el puerto 8080 para
if __name__ == '__main__':
    app.run(port=8080)

---
---

### Lanzo una petición al modelo

In [ ]:
#------ INCLUYO ESTA PETICIÓN EN OTRO NOTEBOOK -------
### Creo el conjunto de datos sobre el que quiero hacer la predicción


import requests

data = {
    "Pclass": 1,
    "Sex": "male",
    "Age": 38,
    "SibSp": 0,
    "Parch": 0,
    "Fare": 71.2833,
    "Embarked": "C"
}

### Hago la petición y obtengo la respuesta
response = requests.post("http://localhost:8080/predigo", json=data)
prediction = response.json()
prediction

#-- El resultado de la predicción es:
import pandas as pd
pred_df = pd.read_json(prediction)